In [0]:
import json
import csv
import tweepy
import re

def search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase):

    #create authentication for accessing Twitter
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    #initialize Tweepy API
    api = tweepy.API(auth)

    #get the name of the spreadsheet we will write to
    fname = '_'.join(re.findall(r"#(\w+)", hashtag_phrase))

    #open the spreadsheet we will write to
    with open('%s.csv' % (fname), 'w') as file:

        w = csv.writer(file)

        #write header row to spreadsheet
        w.writerow(['timestamp', 'tweet_text', 'username', 'all_hashtags', 'followers_count'])

        #for each tweet matching our hashtags, write relevant info to the spreadsheet
        for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', \
                                   lang="en", tweet_mode='extended').items(5000):
            w.writerow([tweet.created_at, tweet.full_text.replace('\n',' ').encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']], tweet.user.followers_count])


consumer_key = 'Snxk4IlXz80CEv5LGtcJ3I52n'
consumer_secret = 'v3vDyMzwNrN9XWsAJro9tJqQTihvRscekmbFHHNUlbdxr5VDa0'
access_token = '1185658874109583366-rcGy5ykUX3SjJMlF1YosHkS2udCrYJ'
access_token_secret = 'wW0zikWScM4RUNprI2nCOIRsDN7lfNvqI7mugTWhqeBiz'

hashtag_phrase = input('Hashtag Phrase ')

if __name__ == '__main__':
    search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase)


In [0]:
import pandas as pd

In [0]:
ca_df = pd.read_csv('ca_labeled.csv', index_col=[0])
ny_df = pd.read_csv('ny_labeled.csv', index_col=[0])
tx_df = pd.read_csv('tx_labeled.csv', index_col=[0])

In [0]:
def features(value):
  if(value == -1):
    value = 'neg'
    return value
  elif(value == 1):
    value = 'pos'
    return value
  else:
    value = 'neu'
    return value

In [0]:
ca_df.head()

,values,tweet_text
0,-1,b'Only some businesses can open this wk. Yes t...
1,-1,b'#California nurse just confirmed they are al...
2,-1,b'I really wish phase 2 included hair and nail...
3,0,b'#California loosens rules for visitor in hos...
4,1,b'This is welcome news for #California. Policy...


In [0]:
ca_df['values'] = ca_df['values'].apply(features)
ny_df['values'] = ny_df['values'].apply(features)
tx_df['values'] = tx_df['values'].apply(features)

ca_df.head()

,values,tweet_text
0,neg,b'Only some businesses can open this wk. Yes t...
1,neg,b'#California nurse just confirmed they are al...
2,neg,b'I really wish phase 2 included hair and nail...
3,neu,b'#California loosens rules for visitor in hos...
4,pos,b'This is welcome news for #California. Policy...


In [0]:
from nltk import TweetTokenizer
tweet_tokenizer = TweetTokenizer()
def tokenize_tweets(tweet):
  tweet = tweet_tokenizer.tokenize(tweet)
  return tweet

In [0]:
ca_df['tweet_text'] = ca_df['tweet_text'].apply(tokenize_tweets)
ny_df['tweet_text'] = ny_df['tweet_text'].apply(tokenize_tweets)
tx_df['tweet_text'] = tx_df['tweet_text'].apply(tokenize_tweets)

In [0]:
ca_df.head()

,values,tweet_text
0,neg,"[b'Only, some, businesses, can, open, this, wk..."
1,neg,"[b, ', #California, nurse, just, confirmed, th..."
2,neg,"[b'I, really, wish, phase, 2, included, hair, ..."
3,neu,"[b, ', #California, loosens, rules, for, visit..."
4,pos,"[b'This, is, welcome, news, for, #California, ..."


In [0]:
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords(tweet):
  tweet = [n for n in tweet if not n in stop_words]
  return tweet

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
ca_df['tweet_text'] = ca_df['tweet_text'].apply(remove_stopwords)
ny_df['tweet_text'] = ny_df['tweet_text'].apply(remove_stopwords)
tx_df['tweet_text'] = tx_df['tweet_text'].apply(remove_stopwords)

ca_df.head()

,values,tweet_text
0,neg,"[b'Only, businesses, open, wk, ., Yes, wanna, ..."
1,neg,"[b, ', #California, nurse, confirmed, saying, ..."
2,neg,"[b'I, really, wish, phase, 2, included, hair, ..."
3,neu,"[b, ', #California, loosens, rules, visitor, h..."
4,pos,"[b'This, welcome, news, #California, ., Policy..."


In [0]:
ca_df.head()

,values,tweet_text
0,neg,"[b'Only, businesses, open, wk, ., Yes, wanna, ..."
1,neg,"[b, ', #California, nurse, confirmed, saying, ..."
2,neg,"[b'I, really, wish, phase, 2, included, hair, ..."
3,neu,"[b, ', #California, loosens, rules, visitor, h..."
4,pos,"[b'This, welcome, news, #California, ., Policy..."


In [0]:
import re
def cleanTxt(text):
    text = re.sub('@[A-Za-z0–9]+', '', text)
    text = re.sub('#', '', text)
    text = re.sub('RT[\s]+', '', text)
    text = re.sub('https?:\/\/\S+', '', text)
    text = re.sub('/', '', text)
    text = text.replace('\\', '')
    
    text = re.sub('x97', '', text)
    text = re.sub('xa3', '', text)
    text = re.sub('x98People', '', text)
    text = re.sub('x98', '', text)
    text = re.sub('xa0', '', text)
    text = re.sub('x94and', '', text)
    text = re.sub('x96', '', text)
    text = re.sub('x99s', '', text)
    text = re.sub('x91', '', text)
    text = re.sub('x8a', '', text)
    text = re.sub('xba', '', text)
    text = re.sub('x9b', '', text)
    text = re.sub('xbc', '', text)
    text = re.sub('x92', '', text)
    text = re.sub('xbf', '', text)
    text = re.sub('x89https', '', text)
    text = re.sub('x94By', '', text)
    text = re.sub('x8f', '', text)
    text = re.sub('xb8', '', text)
    text = re.sub('xa4', '', text)
    text = re.sub('xa5', '', text)
    text = re.sub('x87', '', text)
    text = re.sub('xa5WOW', '', text)
    text = re.sub('x94', '', text)
    text = re.sub('x95', '', text)
    text = re.sub('xb3', '', text)
    text = re.sub('x89', '', text)
    text = re.sub('x9f', '', text)
    text = re.sub('x9ccoronavirus', '', text)
    text = re.sub('xbd', '', text)
    text = re.sub('x9cnatural', '', text)
    text = re.sub('x9cmusic', '', text)
    text = re.sub('xa9', '', text)
    text = re.sub('x82', '', text)
    text = re.sub('xc2', '', text)
    text = re.sub('x83', '', text)
    text = re.sub('x99all', '', text)
    text = re.sub('xb1al', '', text)
    text = re.sub('x9cessential', '', text)
    text = re.sub('x9cEveryone', '', text)
    text = re.sub('x8e', '', text)
    text = re.sub('x98Reopen', '', text)
    text = re.sub('xe3', '', text)
    text = re.sub('xa2', '', text)
    text = re.sub('x80', '', text)
    text = re.sub('x99m', '', text)
    text = re.sub('x90', '', text)
    text = re.sub('x9e', '', text)
    text = re.sub('x99', '', text)
    text = re.sub('xb9', '', text)
    text = re.sub('xbb', '', text)
    text = re.sub('x99re', '', text)
    text = re.sub('xa3https', '', text)
    text = re.sub('x98Burden', '', text)
    text = re.sub('x9cprogressives', '', text)
    text = re.sub('xb1d19', '', text)
    text = re.sub('xaa', '', text)
    text = re.sub('x86', '', text)
    text = re.sub('x8c', '', text)
    text = re.sub('x93', '', text)
    text = re.sub('x9d', '', text)
    text = re.sub('x88', '', text)
    text = re.sub('x99t', '', text)
    text = re.sub('xef', '', text)
    text = re.sub('xf0', '', text)
    text = re.sub('xa7', '', text)
    text = re.sub('xb7', '', text)
    text = re.sub('x9cThe', '', text)
    text = re.sub('x9c', '', text)
    text = re.sub('x99mon', '', text)
    text = re.sub('x99d', '', text)
    text = re.sub('xb5', '', text)
    text = re.sub('xc3', '', text)
    text = re.sub('xe2', '', text)
    text = re.sub('x8d', '', text)
    text = re.sub('xb0', '', text)
    text = re.sub('xa6it', '', text)
    text = re.sub('x98CA', '', text)
    text = re.sub('xc4', '', text)
    text = re.sub('xa8', '', text)
    text = re.sub('x9cthe', '', text)
    text = re.sub('x99ve', '', text)
    text = re.sub('x81', '', text)
    text = re.sub('x8fTake', '', text)
    text = re.sub('x85', '', text)
    text = re.sub('x99S', '', text)
    text = re.sub('xb8OPEN', '', text)
    text = re.sub('xa6', '', text)
    text = re.sub('x8fUplifting', '', text)
    text = re.sub('xb8TYRANT', '', text)
    text = re.sub('xac', '', text)
    text = re.sub('x99ll', '', text)
    text = re.sub('x9cfix', '', text)
    text = re.sub('x98declared', '', text)
    text = re.sub('xa1', '', text)
    text = re.sub('x98fix', '', text)
    return text

In [0]:
ca_df.head()

,values,tweet_text
0,neg,"[b'Only, businesses, open, wk, ., Yes, wanna, ..."
1,neg,"[b, ', #California, nurse, confirmed, saying, ..."
2,neg,"[b'I, really, wish, phase, 2, included, hair, ..."
3,neu,"[b, ', #California, loosens, rules, visitor, h..."
4,pos,"[b'This, welcome, news, #California, ., Policy..."


In [0]:
for i in range(len(ca_df['tweet_text'])):
  for n in range(len(ca_df['tweet_text'][i])):
    ca_df['tweet_text'][i][n] = cleanTxt(ca_df['tweet_text'][i][n])

In [0]:
for i in range(len(ny_df['tweet_text'])):
  for n in range(len(ny_df['tweet_text'][i])):
    ny_df['tweet_text'][i][n] = cleanTxt(ny_df['tweet_text'][i][n])

In [0]:
for i in range(len(tx_df['tweet_text'])):
  for n in range(len(tx_df['tweet_text'][i])):
    tx_df['tweet_text'][i][n] = cleanTxt(tx_df['tweet_text'][i][n])

In [0]:
ca_df.tail()

,values,tweet_text
495,pos,"[b'Thank, , funding, $, 1M, COVID19, relief, g..."
496,neu,"[b, "", California, rework, coronavirus, ethics..."
497,neu,"[b'Is, California, workplace, complying, Cal, ..."
498,neu,"[b'California, around, 4,500, additional, deat..."
499,neu,"[b'California, Governor, temporarily, closes, ..."


In [0]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()

def lemmatize_tweets(tweet):
  tweet = lemmatizer.lemmatize(tweet)
  return tweet

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
for i in range(len(ca_df['tweet_text'])):
  for n in range(len(ca_df['tweet_text'][i])):
    ca_df['tweet_text'][i][n] = lemmatize_tweets(ca_df['tweet_text'][i][n])

for i in range(len(ny_df['tweet_text'])):
  for n in range(len(ny_df['tweet_text'][i])):
    ny_df['tweet_text'][i][n] = lemmatize_tweets(ny_df['tweet_text'][i][n])

for i in range(len(tx_df['tweet_text'])):
  for n in range(len(tx_df['tweet_text'][i])):
    tx_df['tweet_text'][i][n] = lemmatize_tweets(tx_df['tweet_text'][i][n])

In [0]:
import random
#iterate through df and creates tuple for every tweet and every label, append tuples to respective dataset, create a master dataset 
dataset = []

ca_dataset = []
for row in ca_df.itertuples(index = False):
    ca_dataset.append((getattr(row, 'tweet_text'), getattr(row, 'values')))
    
ny_dataset = []
for row in ny_df.itertuples(index = False):
    ny_dataset.append((getattr(row, 'tweet_text'), getattr(row, 'values')))
    
tx_dataset = []
for row in tx_df.itertuples(index = False):
    tx_dataset.append((getattr(row, 'tweet_text'), getattr(row, 'values')))
    
dataset = ca_dataset + ny_dataset + tx_dataset


random.shuffle(dataset)

In [0]:
import nltk

all_words = []
for i in range(len(ca_df.index)):
    for w in ca_df.iat[i, 1]:
        all_words.append(w.lower())
        
for i in range(len(ny_df.index)):
    for w in ny_df.iat[i, 1]:
        all_words.append(w.lower())
        
for i in range(len(tx_df.index)):
    for w in tx_df.iat[i, 1]:
        all_words.append(w.lower())


all_words = nltk.FreqDist(all_words)

In [0]:
word_features = [item[0] for item in all_words.most_common(300)]

In [0]:
def find_features(review):
    words = set(review)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    
    return features

feature_sets = [(find_features(review), rating) for (review, rating) in dataset]

In [0]:
training_set = feature_sets[:1200]
testing_set = feature_sets[1200:]

In [0]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [0]:
print(len(testing_set), len(training_set))

300 1200


In [0]:
results = []
for i in range(len(testing_set)):
    results.append(classifier.classify(testing_set[i][0]))

In [0]:
print(len(results))

300


In [0]:
error = 0
for i in range(len(results)):
    print(results[i], ' : ', testing_set[i][1])
    if(results[i] != testing_set[i][1]):
        error += 1

error = (error/300) * 100
        
print("Error: " + str(error) + "%")

neu  :  neg
neu  :  neu
neu  :  neg
neu  :  neu
pos  :  neu
neu  :  neg
neu  :  neu
neu  :  neu
neu  :  neu
neg  :  neg
pos  :  neu
neu  :  neu
neu  :  neu
neu  :  neg
neu  :  neu
neg  :  neu
pos  :  pos
neu  :  neu
neg  :  neg
neu  :  neu
neu  :  neu
neu  :  neu
pos  :  pos
neu  :  neg
neu  :  neu
neu  :  neu
neu  :  neg
neg  :  neu
neu  :  neu
pos  :  pos
neu  :  neu
pos  :  pos
neu  :  neu
neg  :  neu
neg  :  neg
pos  :  neu
neg  :  neg
neu  :  neu
neu  :  neu
neg  :  neg
neu  :  neg
neu  :  neu
neg  :  neu
pos  :  neu
neg  :  neu
neg  :  neg
neu  :  neg
neu  :  neu
neu  :  neu
neu  :  neu
neu  :  neu
neu  :  neu
neu  :  neg
neu  :  pos
neg  :  neg
pos  :  neu
neg  :  pos
neg  :  neu
neu  :  neu
neu  :  neu
neu  :  pos
neg  :  neg
neu  :  neu
neu  :  neu
neu  :  pos
neu  :  neu
neg  :  neu
neu  :  neg
neu  :  neu
neu  :  neu
neg  :  neg
neu  :  neu
pos  :  neg
pos  :  neu
neu  :  neu
neu  :  neu
neu  :  neu
neu  :  neg
pos  :  pos
neu  :  neu
neu  :  neg
neg  :  pos
neu  :  neu
neu 

In [0]:


ca_new_df = pd.read_csv('COVID_California_current.csv')
ny_new_df = pd.read_csv('COVID19_newyork_current.csv')
tx_new_df = pd.read_csv('COVID19_texas_current.csv')

In [0]:


ca_new_df['tweet_text'] = ca_new_df['tweet_text'].apply(tokenize_tweets)
ny_new_df['tweet_text'] = ny_new_df['tweet_text'].apply(tokenize_tweets)
tx_new_df['tweet_text'] = tx_new_df['tweet_text'].apply(tokenize_tweets)

In [0]:

ca_new_df['tweet_text'] = ca_new_df['tweet_text'].apply(remove_stopwords)
ny_new_df['tweet_text'] = ny_new_df['tweet_text'].apply(remove_stopwords)
tx_new_df['tweet_text'] = tx_new_df['tweet_text'].apply(remove_stopwords)

In [0]:


for i in range(len(ca_new_df['tweet_text'])):
  for n in range(len(ca_new_df['tweet_text'][i])):
    ca_new_df['tweet_text'][i][n] = cleanTxt(ca_new_df['tweet_text'][i][n])

for i in range(len(ny_new_df['tweet_text'])):
  for n in range(len(ny_new_df['tweet_text'][i])):
    ny_new_df['tweet_text'][i][n] = cleanTxt(ny_new_df['tweet_text'][i][n])

for i in range(len(tx_new_df['tweet_text'])):
  for n in range(len(tx_new_df['tweet_text'][i])):
    tx_new_df['tweet_text'][i][n] = cleanTxt(tx_new_df['tweet_text'][i][n])

In [0]:


for i in range(len(ca_new_df['tweet_text'])):
  for n in range(len(ca_new_df['tweet_text'][i])):
    ca_new_df['tweet_text'][i][n] = lemmatize_tweets(ca_new_df['tweet_text'][i][n])

for i in range(len(ny_new_df['tweet_text'])):
  for n in range(len(ny_new_df['tweet_text'][i])):
    ny_new_df['tweet_text'][i][n] = lemmatize_tweets(ny_new_df['tweet_text'][i][n])

for i in range(len(tx_new_df['tweet_text'])):
  for n in range(len(tx_new_df['tweet_text'][i])):
    tx_new_df['tweet_text'][i][n] = lemmatize_tweets(tx_new_df['tweet_text'][i][n])

In [0]:
import nltk

all_words_new = []
for i in range(len(ca_new_df.index)):
    for w in ca_new_df.iat[i, 1]:
        all_words_new.append(w.lower())
        
for i in range(len(ny_new_df.index)):
    for w in ny_new_df.iat[i, 1]:
        all_words_new.append(w.lower())
        
for i in range(len(tx_new_df.index)):
    for w in tx_new_df.iat[i, 1]:
        all_words_new.append(w.lower())


all_words_new = nltk.FreqDist(all_words)

In [0]:

word_features_new = [item[0] for item in all_words_new.most_common(300)]

In [0]:


results_new_ca = []

for i in range(len(ca_new_df['tweet_text'])):
  feature = find_features(ca_new_df['tweet_text'][i])
  results_new_ca.append(classifier.classify(feature))

results_new_ny = []

for i in range(len(ny_new_df['tweet_text'])):
  feature = find_features(ny_new_df['tweet_text'][i])
  results_new_ny.append(classifier.classify(feature))

results_new_tx = []

for i in range(len(tx_new_df['tweet_text'])):
  feature = find_features(tx_new_df['tweet_text'][i])
  results_new_tx.append(classifier.classify(feature))

In [0]:
def sentiment_percent(results):
  neg = 0
  neu = 0
  pos = 0
  for i in results:
    if(i == 'neg'):
      neg += 1
    elif(i == 'neu'):
      neu += 1
    else:
      pos += 1

  print("Pos:", str(pos / len(results) * 100) + "%")
  print("Neg:", str(neg / len(results) * 100) + "%")
  print("Neu:", str(neu / len(results) * 100) + "%")

In [0]:
sentiment_percent(results_new_ca)

Pos: 5.544147843942506%
Neg: 21.149897330595483%
Neu: 73.30595482546201%


In [0]:
sentiment_percent(results_new_ny)

Pos: 7.8125%
Neg: 17.038690476190478%
Neu: 75.14880952380952%


In [0]:
sentiment_percent(results_new_tx)

Pos: 7.452339688041594%
Neg: 29.116117850953206%
Neu: 63.43154246100519%


https://www.youtube.com/watch?v=ujId4ipkBio

https://gist.github.com/vickyqian/f70e9ab3910c7c290d9d715491cde44c

https://towardsdatascience.com/twitter-sentiment-analysis-classification-using-nltk-python-fa912578614c

https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f

https://medium.com/@mr.adam.maj/machines-key-to-understanding-humans-how-i-used-natural-language-processing-to-analyze-human-9745d04e534b